In [1]:
import functools
from collections import namedtuple
param_struct = namedtuple('param_struct', ['args', 'kwargs'])
def tco(func):
    def call(*args, **kwargs):
        try:
            old = func.__globals__[func.__name__] 
            func.__globals__[func.__name__] = lambda *args, **kwargs : param_struct(args, kwargs)
            res = param_struct(args, kwargs)
            while True:
                res = func(*res.args, **res.kwargs)
                if isinstance(res, param_struct):
                    continue
                break
            func.__globals__[func.__name__] = old
            return res
        except Exception as e:
            func.__globals__[func.__name__] = old
            raise e
    return call


In [2]:
@tco
def fact(n, out=0):
    if n is 1:
        return out
    else:
        return fact(n-1 , out = out*n)          
def fact2(n):
    return fact2(n-1)*n if n is not 1 else 1
def fact3(n):
    from functools import reduce
    return reduce(lambda x,y: x*y, range(1, n+1))

In [3]:

import timeit
import sys
sys.setrecursionlimit(10500)
for func in (fact, fact2, fact3):
    print(func.__name__)
    %timeit func(10000)

call
9.14 ms ± 302 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
fact2
20.9 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
fact3
16 ms ± 464 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
